# Troubleshooting

Eventually, all code will fail. Even GETTSIM's and your code is no exception. Therefore, GETTSIM offers a debug mode for the function `compute_taxes_and_transfers` which helps you to find the source of the error.

Let us take the same example as used in the [tutorial on basic usage](basic_usage.ipynb), but reduced to the computation of `ges_rentenv_beitr_m` which are the monthly contribution to the pension insurance.

In [9]:
import pandas as pd
from gettsim import compute_taxes_and_transfers
from gettsim import set_up_policy_environment

In the following cell, we create all necessary data to run the example.

In [2]:
data = pd.DataFrame(
    {
        "p_id": 1,
        "tu_id": 1,
        "hh_id": 1,
        "bruttolohn_m": 1000,
        "wohnort_ost": False,
    },
    index=[0],
)
data

,p_id,tu_id,hh_id,bruttolohn_m,wohnort_ost
0,1,1,1,1000,False


In [3]:
policy_params, policy_functions = set_up_policy_environment(2020)

In the following cell, we call the function to compute quantities in the tax and transfer system. Note that, `debug = True` which enables the debug mode. It is `False` by default. The debug mode differs from the usual call in two ways:

1. All necessary inputs to compute the `targets` and all intermediate variables are returned. This lets you assess your inputs and outputs together which can be useful to identify issues which may be preventing certain variables from being computed.
2. If an exception occurs while computing the targets, the exception is printed, but not raised. The variables which depend on the variable where the exception occurred are not computed.

The following call shows the effect of the debug mode when no exception occurs.

In [4]:
df = compute_taxes_and_transfers(
    data=data,
    functions=policy_functions,
    params=policy_params,
    targets="ges_rentenv_beitr_m",
    debug=True,
)

The resulting DataFrame contains all necessary input variables as well as the target and intermediate variables.

In [5]:
df

,hh_id,tu_id,p_id,bruttolohn_m,wohnort_ost,regulär_beschäftigt,_ges_rentenv_beitr_bemess_grenze_m,mini_job_grenze,_ges_rentenv_beitr_bruttolohn_m,geringfügig_beschäftigt,in_gleitzone,midi_job_bemessungsentgelt_m,_ges_rentenv_beitr_midi_job_m_m,ges_rentenv_beitr_m
0,1,1,1,1000,False,False,6900.0,450.0,NaN,False,True,961.040588,85.753549,85.753549


## Debug Mode with an Exception

What happens when an exception is raised while producing the target? We artificially produce an error with a modified user function. For exemplary purposes we modify GETTSIM's function `mini_job_grenze` which is called when computing the pension contributions. Instead of its usual functionality of returning the income threshold for marginal employment, we overwrite it to raise an error when called (note that we could do so with any other function as well).

In [6]:
def mini_job_grenze():
    raise ValueError

In [7]:
df = compute_taxes_and_transfers(
    data=data,
    functions=[policy_functions, mini_job_grenze],
    params=policy_params,
    targets="ges_rentenv_beitr_m",
    debug=True,
)

Traceback (most recent call last):
  File "C:\ECON\gettsim\gettsim\dag.py", line 279, in execute_dag
    results[task] = dag.nodes[task]["function"](**kwargs).rename(task)
  File "C:\Users\Christian Zimpelmann\AppData\Local\Temp\ipykernel_38460\2525973380.py", line 2, in mini_job_grenze
    raise ValueError
ValueError


The traceback of the error is printed, but the execution is continued. The resulting DataFrame captures all input variables as well as variables which could be produced without errors.

In [8]:
df

,hh_id,tu_id,p_id,bruttolohn_m,wohnort_ost,regulär_beschäftigt,_ges_rentenv_beitr_bemess_grenze_m,_ges_rentenv_beitr_bruttolohn_m
0,1,1,1,1000,False,False,6900.0,NaN
